In [42]:
import  pandas as pd
data_path = 'data/'
re_files = ['sqd_kre_.csv', 'lidongdong/lidongdong_kre.csv']
re_k = pd.read_csv(data_path+re_files[0])
for item in re_files[1:]:
    re_k = pd.concat([re_k, pd.read_csv(data_path+item)], ignore_index=True)

/home/shiqundong/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [75]:
sms_data = pd.read_excel(data_path+'北京中移在线/智能短信模板数据20180827.xls', sheet_name='流量类')
sence_key = pd.read_table(data_path+'lidongdong/template_Phone_key_with_scene.txt', header=None, names=['sence','key'])
re_l = pd.read_csv(data_path+'sqd_kre_list.csv')

In [168]:
def get_list_kv(re_path, sentece):
    wanted = {}
    re_expressions = pd.read_csv(re_path)['re'].values
    k = 1
    for re_expression in re_expressions:
        finds = re.findall(re_expression, sentece)
        if finds:
            for find in finds:
                name1 = '套餐{}名称'.format(k)
                name2 = '套餐{}已使用'.format(k)
                name3 = '套餐{}还剩余'.format(k)
                k += 1
                wanted.update({name1:find[0],\
                              name2:find[1],\
                              name3:find[2]})
    return wanted

def get_kv(re_path, setence):
    wanted = []
    re_csv = pd.read_csv(re_path)
    for i in re_csv.index:
        re_expression = re_csv.loc[i]['re']
        find = re.search(re_expression, setence)
        if find:
            key = re_csv.loc[i]['key']
            wanted.append({'find': find, \
                           'key': key, \
                           'cn_key': re_csv.loc[i]['cn_key'], \
                           'value': find.group(key) } )
    return wanted
    
def get_result(sentece):
    #the output is (:tranffic_name , :tranffic_used, tranffic_balance) dict object
    list_kv = get_list_kv('data/sqd_kre_list.csv', sentece)
    #the output is [{find:, key:, cn_key:, value:}]  dict list
    kv = get_kv('data/kre.csv', sentence)
    if len(kv) == 0:
        return list_kv
    kv_df = pd.DataFrame(kv)
    
    kv_df = kv_df.drop_duplicates(['cn_key', 'value'])
    
    cn_key_list = kv_df['cn_key'].values
    if len(cn_key_list) == len(set(cn_key_list)):
        re_r =  dict(zip(kv_df['cn_key'].values, kv_df['value'].values))
        re_r.update(list_kv)
        return re_r
    
    #find min length find when cn_key is same
    def get_minlen_find(df):
        find_length = [len(str(item)) for item in df['find'].values]
        df['find_length'] = find_length
        min_len = min(find_length)
        return df[df.find_length == min_len]
    kv_df= kv_df.groupby(['cn_key']).apply(get_minlen_find)

    cn_key_list = kv_df['cn_key'].values
    if len(cn_key_list) == len(set(cn_key_list)):
        re_r =  dict(zip(kv_df['cn_key'].values, kv_df['value'].values))
        re_r.update(list_kv)
        return re_r
    
    #find min length value when cn_key is same
    def get_minlen_value(df):
        find_length = [len(str(item)) for item in df['value'].values]
        df['value_length'] = find_length
        min_len = min(find_length)
        return df[df.find_length == min_len]
    kv_df= kv_df.groupby(['value']).apply(get_minlen_value)
    
    
    re_r =dict(zip(kv_df['cn_key'].values, kv_df['value'].values))
    re_r.update(list_kv)
    return re_r

In [170]:

for i in sms_data.index:
    sentence = sms_data.loc[i]['短信原文']
    if sentence.find('流量使用情况')!=-1:
        print('='*80+'\n%s\n'%sentence)
        result = get_result(sentence)
        for k,v in result.items():
            print('-'*50+'\n'+'{}:{}\n'.format(k, v))
        

【江西移动流量提醒服务】您好！截至2016年12月9日10时41分，您当月套餐外数据流量费用（不含国际及港澳台漫游）已超过10元（仅供参考，具体以月结账单为准）。更多流量使用情况可点击wap.jx.10086.cn/ll。中国移动

--------------------------------------------------
截止时间:2016年12月9日10时41分

【流量超量提醒】您好！截至03月15日22时39分，您本月套餐及获赠的国内通用流量已用完，为避免您本月产生套餐外流量，建议您回复“810”办理10元包100MB加油包，当月生效，下月自动取消，更多流量使用情况可点击http://wap.hn.10086.cn/llcx.jsp。中国移动

--------------------------------------------------
截止时间:03月15日22时39分

--------------------------------------------------
赠送流量:的国内通用流量已用完，为避免您本月产生套餐外流量，建议您回复“810”办理10元包100M

--------------------------------------------------
流量状态:已用完

【流量超量提醒】您好！截至09月19日01时15分，您本月已使用移动数据流量1GB395MB，套餐及获赠的国内通用流量已用完，超出套餐的流量费用已达5.40元，为节省您本月的上网费用，建议您回复8101办理10元包100MB加油包（回复810可选择办理更多档次加油包），当月办理立即生效（从4月1日起，加油包不能再抵扣订购前您产生的移动数据流量），仅限当月使用，下月自动取消，更多流量使用情况可点击http://wap.hn.10086.cn/llcx.jsp。中国移动

--------------------------------------------------
截止时间:09月19日01时15分

--------------------------------------------------
已使用流量:1G

--------------------------------------------------
赠送流量:的国内通用流

--------------------------------------------------
截止时间:13日18时58分

--------------------------------------------------
国内通用流量:流量511.46M

--------------------------------------------------
流量共有:1G

--------------------------------------------------
剩余流量:流量511.46M

【流量提醒】您本月国内移动数据流量共有1GB，截至5日14时27分，剩余流量503.67MB，供您参考。如您办有假日包、夜间包、快餐包等专享套餐，详细流量使用情况请点击：http://wap.js.10086.cn/dz。

--------------------------------------------------
截止时间:5日14时27分

--------------------------------------------------
国内通用流量:流量503.67M

--------------------------------------------------
流量共有:1G

--------------------------------------------------
剩余流量:流量503.67M

【流量提醒】您本月国内移动数据流量共有30.00MB，截至1日7时51分，已使用完，供您参考。如您办有假日包、夜间包、快餐包等专享套餐，详细流量使用情况请点击：http://wap.js.10086.cn/dz。

--------------------------------------------------
截止时间:1日7时51分

--------------------------------------------------
流量共有:30.00M

【流量提醒】您本月国内移动数据流量共有30.00MB，截至6日22时24分，已使用完，供您参考。此外，您本月还剩余省内4G单模流量498.53M。如您办有假日包、夜间包、快餐包等专享套餐，详细流量使用情况请点击：http://wap.js.10086

--------------------------------------------------
已使用流量:1G

【流量提醒】尊敬的客户，截至06月26日16时55分，您本月套餐内国内流量已用完，建议您回复KTAXB办理流量安心包，办理后套餐外流量费用先按0.29元/MB收取，当费用达到10元时，可使用流量至100MB，当再次超出后,按同样规则以此类推，当月办理立即生效，套餐不退订一直有效。为节约您的上网费用，建议您回复88202升级为20元包150MB流量套餐，下月生效。查询流量使用情况可拨打1008611。【中国移动】

--------------------------------------------------
截止时间:06月26日16时55分

--------------------------------------------------
已使用流量:100M

--------------------------------------------------
流量状态:已用完

【流量提醒】尊敬的客户，截至08月04日11时56分，您本月已使用移动数据流量80.31MB，套餐内流量剩余99.69MB，查询流量使用情况可拨打1008611。【中国移动】

--------------------------------------------------
截止时间:08月04日11时56分

--------------------------------------------------
已使用流量:80.31M

--------------------------------------------------
剩余流量:99.69M

--------------------------------------------------
已使用:80.31MB

--------------------------------------------------
剩余:99.69MB

【流量提醒服务】您好！您本月流量统付流量叠加包500M数据流量共有500MB，截至2017年5月5日9时0分21秒，您本月该套餐内国内（不含港澳台）流量已用尽， 更多流量使用情况可点击wap.jx.10086.cn/ll。【中国移动】

--


--------------------------------------------------
套餐2名称:省内流量套餐

--------------------------------------------------
套餐2已使用:20.29M

--------------------------------------------------
套餐2还剩余:14622.91M

--------------------------------------------------
套餐3名称:新68优惠10元100M国内流量

--------------------------------------------------
套餐3已使用:0M

--------------------------------------------------
套餐3还剩余:100.00M

--------------------------------------------------
套餐4名称:新2元超级叠加包(100M国内流量)

--------------------------------------------------
套餐4已使用:0M

--------------------------------------------------
套餐4还剩余:100.00M

话费、流量、账单、积分等信息查询请点击：http://dx.10086.cn/RnyqmIj  一、话费使用情况： 您为合帐账户截止03月01日10时30分，您当月合帐账户话费已消费27.97元，话费余额4214.19元； 二、流量使用情况: 上月结转至本月的国内高速流量套餐为2048.00M。您的国内高速流量套餐已使用0MB，还剩余2548.00M（含结转流量2048.00M)；80元国内流量不限量优惠包(国内流量不限量+赠送200分钟国内+赠送1400分钟本地)已使用29.37M，还剩余25570.63M。 【开学季，海量流量免费送！】1024M流量见面礼+5亿M流量齐瓜分，尽在“中国移动10086”微信公众号，详情点击http://dx.10086.cn/JFrmiq 【中国移动】

---------------------------------------

--------------------------------------------------
截止时间:11月01日08时59分

--------------------------------------------------
已使用流量:0M

--------------------------------------------------
国内通用流量:300.00M

--------------------------------------------------
剩余流量:300.00M

--------------------------------------------------
话费余额:39.65元

--------------------------------------------------
当前话费:103.77元

--------------------------------------------------
**流量名字:您的国内流量套餐

--------------------------------------------------
**流量已使用:0M

--------------------------------------------------
**流量剩余:300.00M

--------------------------------------------------
消费金额:103.77元

--------------------------------------------------
已使用:0MB

--------------------------------------------------
剩余:300.00M

--------------------------------------------------
套餐1名称:您的国内流量套餐

--------------------------------------------------
套餐1已使用:0M

--------------------------------------------------
套餐1还剩余:300.00M

话费、流量、账单、积分等信息查询请点击：http:/

--------------------------------------------------
截止时间:02月28日23时59分

--------------------------------------------------
已使用流量:1255.00M

--------------------------------------------------
话费余额:48.32元

--------------------------------------------------
当前话费:111.68元

--------------------------------------------------
消费金额:111.68元

--------------------------------------------------
流量安心包:1255.00M

--------------------------------------------------
流量状态:已用完

话费、流量、账单信息查询请点击：http://dx.10086.cn/aeM7NVr进行查看。一、话费使用情况：截止03月02日00时03分，您当月话费已消费4.92元，话费余额50.37元；二、流量使用情况:上月结转至本月的国内高速流量套餐为61.60M。您的国内高速流量套餐已使用29.55M，还剩余132.05M，（含结转流量32.05M)。【开学季，海量流量免费送！】1024M流量见面礼+5亿M流量齐瓜分，尽在“中国移动10086”微信公众号，详情点击http://dx.10086.cn/JFrmiq【中国移动】

--------------------------------------------------
上月不清零国内流量:的国内高速流量套餐为61.60M

--------------------------------------------------
剩余:132.05M

--------------------------------------------------
剩余流量:132.05M

--------------------------------------------------
国内通用流量:132.05M

---


--------------------------------------------------
**流量名字:省内高速流量套餐

--------------------------------------------------
**流量已使用:14548.08M

--------------------------------------------------
上月不清零国内流量:的国内高速流量套餐为500.00M

--------------------------------------------------
剩余:1000.00M

--------------------------------------------------
剩余流量:1000.00M

--------------------------------------------------
国内通用流量:1000.00M（含结转流量500.00M)

--------------------------------------------------
已使用:0MB

--------------------------------------------------
已使用流量:0M

--------------------------------------------------
当前话费:92.30元

--------------------------------------------------
截止时间:04月30日13时20分

--------------------------------------------------
消费金额:92.30元

--------------------------------------------------
话费余额:117.75元

--------------------------------------------------
套餐1名称:您的国内高速流量套餐

--------------------------------------------------
套餐1已使用:0M

--------------------------------------------------
套餐1

--------------------------------------------------
已使用流量:2G

鉴于您上月的流量使用情况，为了匹配您的实际流量用量以及消费成本，建议您考虑改为30元包500M流量包月，次（当）月生效。

--------------------------------------------------
已使用流量:500M

流量超量提醒：您好！截至 6月 8日13时55分，您本月已使用移动数据流量1GB47.03MB，套餐外流量费用已达13.63元，流量王流量剩余1GB0.00MB。套餐外流量资费为0.29元/MB。回复18查询本月移动数据流量使用情况，回复742了解更多移动数据流量套餐信息，回复QXLLTX取消流量提醒。【中国移动】

--------------------------------------------------
截止时间:8日13时55分

--------------------------------------------------
已使用流量:1G

--------------------------------------------------
剩余流量:1G

流量超量提醒：您好！截至 7月 2日 2时 0分，您本月已使用移动数据流量1GB739.20MB，套餐外流量费用已达100.36元。套餐外流量资费为0.29元/MB。回复18查询本月移动数据流量使用情况，回复742了解更多移动数据流量套餐信息，回复QXLLTX取消流量提醒。【中国移动】

--------------------------------------------------
截止时间:2日

--------------------------------------------------
已使用流量:1G

流量超量提醒：您好！截至1月5日16时42分，您本月已使用移动数据流量1GB251.95MB，套餐外流量费用已达104.06元，省内流量剩余4GB176.00MB，省内闲时流量剩余1000.00MB。套餐外流量资费为0.29元/MB。回复18查询本月移动数据流量使用情况，回复742了解更多移动数据流量套餐信息，回复QXLLTX取消流量提醒。【中国移动】

----------------------

--------------------------------------------------
截止时间:2018年01月17日15时55分

--------------------------------------------------
已使用流量:21G

--------------------------------------------------
剩余流量:流量可滚存下月）：139邮箱（VIP版）本月内含10M

您好！截止2018年01月18日11时16分，您各类套餐使用情况如下：本月流量：按月计费流量产品（部分套餐内含剩余流量可滚存下月）：流量安心包尚未使用；4G飞享28套餐B（2016标准版）本月内含300MB，已使用233MB，剩余67MB；按固定时长计费的产品（限有效期内使用，过期失效）：您的60元6GB流量降费红包内含6GB，已使用308MB（本月未使用），剩余5GB716MB，有效期截止2018年02月22日。回复“28*月份#”查询相应月份套餐流量使用情况；回复“CXJZLL”查询流量不清零套餐列表。【中国移动】

--------------------------------------------------
截止时间:2018年01月18日11时16分

--------------------------------------------------
已使用流量:4G

--------------------------------------------------
剩余流量:流量可滚存下月）：流量安心包尚未使用；4G

--------------------------------------------------
已使用:233MB

--------------------------------------------------
剩余:67MB

您好！截止2018年01月20日11时28分，您各类套餐使用情况如下：上月结转流量：国内流量为118MB，已使用完。本月流量：按月计费流量产品（部分套餐内含剩余流量可滚存下月）：4G飞享38套餐B（2016标准版）本月内含500MB，已使用184MB，剩余316MB。回复“28*月份#”查询相应月份套餐流量使用情况；回复“CXJZLL”查询流量不清零套

--------------------------------------------------
已使用流量:135M

尊敬的客户，截止2017年6月25日 24时您的流量使用情况如下：4G飞享套餐-128元B套餐-国内流量已使用2048.00M，剩余0.00M。30元手机上网流量包-国内流量 已使用500.00M，剩余0.00M。

--------------------------------------------------
截止时间:2017年6月25日

--------------------------------------------------
已使用流量:4G

--------------------------------------------------
国内通用流量:0.00M

--------------------------------------------------
剩余流量:0.00M

--------------------------------------------------
已使用:2048.00M

--------------------------------------------------
剩余:0.00M

尊敬的客户，截至${END_DATE}，您已使用50%的套餐流量，回复115可随时查流量使用情况。回

尊敬的客户:截至07月19日20点，您本月使用的国内上网数据流量有2GB123.9MB。您本月可享受国内数据流量166.6MB（其中上月结转66.6MB），已使用0.7MB，剩余可用165.9MB；可享受省内3/4G赠送流量3GB，已使用2GB123.1MB，剩余可用900.9MB；可享受赠送全国流量50.0MB，已使用0MB，剩余可用50.0MB；更多流量使用情况可点击 http://t.cn/Ryu3Zbd 。中国移动

--------------------------------------------------
剩余流量:可用165.9M

--------------------------------------------------
国内通用流量:可用165.9M

--------------------------------------------

ValueError: Invalid call for scalar access (getting)!

In [83]:
pd.DataFrame([{'a':1,'b':2},{'a':2, 'b':3}])

,a,b
0,1,2
1,2,3
